In [10]:
import os
import sys
import importlib

import pandas as pd
import numpy as np
import sklearn.tree as tree
import sklearn.model_selection as sel
import sklearn.metrics as metrics
import xgboost.sklearn as xgb
import seaborn as sb

from sklearn import svm


# add project modules to the path
path_to_module = os.path.abspath(os.path.join(os.getcwd(),"..","src/"))
sys.path.append(path_to_module)

import src.models.train_model as train

In [41]:
xgb.XGBClassifier(random_state=42,
                 verbosity=2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=2)

I'm now repeating code a lot here, bad! Will need to write bits of this

In [2]:
path_to_data = os.path.abspath(os.path.join(os.getcwd(),
                                            "..",
                                            "data/processed/"
                                           ))

data_dict = train.load_processed_data(file_path=path_to_data)

loading files from /home/edd/Documents/Projects/minst-ensemble/data/processed: 100%|██████████| 4/4 [00:01<00:00,  2.02it/s]


In [3]:
X = data_dict["X_train"]
y = data_dict["y_train"]

In [4]:
X_train, X_test, y_train, y_test = sel.train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [5]:
X_train_smol, X_test_smol, y_train_smol, y_test_smol = sel.train_test_split(X,
                                                    y,
                                                    test_size=0.09,
                                                    train_size=0.21,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [47]:
xgb_clf = xgb.XGBClassifier(random_state=42,
                            verbosity=1,
                            n_jobs=2)

In [21]:
xgb_params = {"n_estimators": [100,200,500],
              "learning_rate": [0.01, 0.1, 1],
              "max_depth": [3,5,10]
             }
            

In [22]:
grid_search = sel.GridSearchCV(estimator=xgb_clf, 
                               param_grid=xgb_params,
                               scoring = "f1_macro",
                               n_jobs = 2,
                               cv=3,
                               verbose = 10
                              )

In [23]:
grid_search.fit(X_train_smol, y_train_smol)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  2.0min
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  6.0min
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed: 23.8min
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed: 37.8min
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed: 80.9min
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 149.2min
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 186.7min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 231.3min
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed: 295.7min
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed: 477.6min
[Parallel(n_jobs=2)]: Done  81 out of  81 | elapsed: 546.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=2,
                                     nthread=None, objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=2,
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 500]},
  

In [24]:

grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500}

In [25]:
grid_search.best_score_

0.9372817140536147

interesting that this is similar to extratrees and randomforest, seems tree based methods can't get over 0.94.

Like the beaver building a dam, we can always thow more trees at the problem

In [32]:
xgb_params = {"n_estimators": [1000, 2000],
              "learning_rate": [0.01, .1, 1],
              "max_depth": [5,10]
             }

In [48]:
grid_search = sel.GridSearchCV(estimator=xgb_clf, 
                               param_grid=xgb_params,
                               scoring = "f1_macro",
                               n_jobs = 2,
                               cv=3,
                               verbose = 10
                              )

In [ ]:
grid_search.fit(X_train_smol, y_train_smol)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
